# Day1 实验：从 Prompt 到 Action

本实验学习如何创建基础 Google ADK Agent，实现从简单提示词到智能行动的转换。

**学习目标：**
- 配置 Gemini API 环境
- 创建第一个 Agent
- 使用 Google Search 工具
- 理解 Agent 运行机制

## 步骤 1: 配置 API Key
从项目根目录的 `.env` 文件读取 `GOOGLE_API_KEY`

In [9]:
import os
from pathlib import Path

# 读取项目根目录的 .env 文件
env_file = Path.cwd().parent / '.env'

if env_file.exists():
    for line in env_file.read_text().splitlines():
        if line.startswith('GOOGLE_API_KEY='):
            os.environ["GOOGLE_API_KEY"] = line.split('=', 1)[1].strip()
            print("✅ Gemini API key setup complete.")
            break
else:
    print(f"⚠️ Please create .env file at: {env_file}")


✅ Gemini API key setup complete.


## 导入ADK 组件
现在，从Agent Development Kit和Generative AI库中导入你需要的特定组件。这能让你的代码保持条理清晰，并确保我们能够访问必要的构建模块。

In [10]:
from google.adk.agents import Agent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.tools import google_search
from google.genai import types

print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


## 步骤 3: 配置重试策略
设置 HTTP 请求的重试选项，确保 API 调用稳定性

In [11]:
retry_config=types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1, # Initial delay before first retry (in seconds)
    http_status_codes=[429, 500, 503, 504] # Retry on these HTTP errors
)

## 步骤 4: 创建 Agent
定义一个具备 Google Search 能力的助手 Agent

In [12]:
root_agent = Agent(
    name="helpful_assistant",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    description="A simple agent that can answer general questions.",
    instruction="You are a helpful assistant. Use Google Search for current info or if unsure.",
    tools=[google_search],
)

print("✅ Root Agent defined.")

✅ Root Agent defined.


## 步骤 5: 创建 Runner
Runner 负责执行 Agent 的推理和工具调用流程

In [13]:
runner = InMemoryRunner(agent=root_agent)

print("✅ Runner created.")

✅ Runner created.


## 步骤 6: 测试 Agent

### 测试 1: 历史事件查询
Agent 会自动判断是否需要使用 Google Search

In [14]:
response = await runner.run_debug(
    "Who won the FIFA World Cup in 2022? answer in Chinese."
)


 ### Created new session: debug_session_id

User > Who won the FIFA World Cup in 2022? answer in Chinese.
helpful_assistant > 2022年国际足联世界杯的冠军是阿根廷队。在决赛中，阿根廷队通过点球大战以4-2战胜了法国队，获得了他们队史上的第三个世界杯冠军。
helpful_assistant > 2022年国际足联世界杯的冠军是阿根廷队。在决赛中，阿根廷队通过点球大战以4-2战胜了法国队，获得了他们队史上的第三个世界杯冠军。


### 测试 2: 实时信息查询
观察 Agent 如何调用 Google Search 获取实时天气信息

In [15]:
response = await runner.run_debug("What's the weather in shenzhen?")


 ### Continue session: debug_session_id

User > What's the weather in shenzhen?
helpful_assistant > Currently in Shenzhen, it is partly cloudy with a temperature of 82°F (28°C). The humidity is around 43%.

Here's a brief forecast for the next few days:
*   **Today (Saturday, November 15, 2025):** Partly cloudy during the day and partly cloudy at night. The temperature will be between 69°F (21°C) and 83°F (28°C).
*   **Sunday, November 16, 2025:** Sunny during the day and clear with periodic clouds at night. The temperature will be between 69°F (21°C) and 79°F (26°C).
*   **Monday, November 17, 2025:** Cloudy during the day and partly cloudy at night. The temperature will be between 64°F (18°C) and 83°F (28°C).

Shenzhen generally has a humid, subtropical climate with warm, damp summers and mild, relatively dry winters.
helpful_assistant > Currently in Shenzhen, it is partly cloudy with a temperature of 82°F (28°C). The humidity is around 43%.

Here's a brief forecast for the next few

## 扩展：Web UI 界面

**方式 1：** 命令行启动
```bash
uv run adk ui
```

**方式 2：** 使用提供的 Web UI 脚本
```bash
uv run python day1-01-First-Agent-Web-UI.py
```
访问 http://localhost:8000 使用可视化聊天界面

**提示：** `run_debug()` 已提供详细执行日志，适合学习和调试

In [16]:
# 🎉 实验总结
# ✅ 成功配置 Gemini API
# ✅ 创建具备搜索能力的 Agent
# ✅ 理解 Agent 的工具调用机制
# ✅ 掌握 run_debug 的使用方法

print("🎉 Day1 实验完成！")
print("📝 下一步: 尝试添加更多工具，或修改 instruction 观察 Agent 行为变化")

🎉 Day1 实验完成！
📝 下一步: 尝试添加更多工具，或修改 instruction 观察 Agent 行为变化


## Agent 工作流程示意图

![Agent Workflow](https://fisherai-1312281807.cos.ap-guangzhou.myqcloud.com/202511151516707.png)